In [8]:
import warnings
warnings.filterwarnings(action='ignore')

import os
import pandas as pd
import numpy as np

from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
import IPython.display as ipd
import time

In [9]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torch
import torchvision
import torch.nn.functional as F
from torchvision.transforms import functional
from torch import Tensor, nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid
import torchvision.transforms as transforms
import albumentations as A
from albumentations.pytorch import ToTensor

from efficientnet_pytorch import EfficientNet

In [44]:
import torch
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models

## 데이터 불러오기

In [45]:
train = pd.read_csv('./train/train_data.csv')
test = pd.read_csv('./test/test/test_data.csv')

In [78]:
train.columns = ['file_name','label']

In [65]:
test.head()

,file_name
0,idx0001.png
1,idx0002.png
2,idx0003.png
3,idx0004.png
4,idx0005.png


In [106]:
class CustomDataset(torch.utils.data.Dataset): 
    def __init__(self,df,path,option,augmentation=None):
        self.df = df
        self.option = option
        self.augmentation = augmentation
        self.path = path

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx): 
        file_path  = self.df.loc[idx,'file_name']
        image=  Image.open(os.path.join(self.path,file_path)).convert('RGB')
        image = np.array(image)
        
        #image = transforms.ToTensor()(image=image)['image']
        
        
        if self.augmentation is not None:
            image = self.augmentation(image=image)['image']
        
        
        if self.option =='train':
            label = self.df.loc[idx,'label']
            label = torch.tensor(label, dtype=torch.int64)
            return image, label
        
        return image
    

In [113]:
train_transforms = A.Compose([
    A.Normalize(),

    A.HorizontalFlip(),
    A.RandomRotate90(p=1),
    A.VerticalFlip(p=1),
    ToTensor()
])

test_transforms = A.Compose([
    A.Normalize(),
    ToTensor()
])

In [114]:


train_dataset = CustomDataset(train,'train/','train',train_transforms)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)

In [119]:
class MnistModel(nn.Module):
    def __init__(self,):
        super().__init__()
        self.model = models.resnet101(pretrained=False)
        
        self.dropout = nn.Dropout(0.5)
        
        self.classifier = nn.Linear(1000, 10)

    def forward(self, images):
        outputs = self.model(images)
        outputs = self.dropout(outputs)
        outputs = self.classifier(outputs)
        return outputs

In [120]:
model = MnistModel().to('cuda')

In [121]:
optimizer = optim.Adam(model.parameters(), lr=1e-4, )
criterion = nn.CrossEntropyLoss()

In [122]:
num_epochs = 100
model.train()
for epoch in range(num_epochs):
    for i, (images, targets) in enumerate(train_loader):
        images, targets = images.to('cuda'), targets.to('cuda')

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()  
        
        if (i+1) % 20 == 0:
            print(f'Epoch: {epoch} - Loss: {loss:.6f}')

Epoch: 0 - Loss: 2.347084
Epoch: 0 - Loss: 2.526636
Epoch: 0 - Loss: 2.563997
Epoch: 1 - Loss: 2.491219
Epoch: 1 - Loss: 2.397850
Epoch: 1 - Loss: 2.408494
Epoch: 2 - Loss: 2.452332
Epoch: 2 - Loss: 2.450929
Epoch: 2 - Loss: 2.507889
Epoch: 3 - Loss: 2.420635
Epoch: 3 - Loss: 1.983325
Epoch: 3 - Loss: 2.376291
Epoch: 4 - Loss: 2.534738
Epoch: 4 - Loss: 2.015482
Epoch: 4 - Loss: 1.949870
Epoch: 5 - Loss: 2.296866
Epoch: 5 - Loss: 2.351448
Epoch: 5 - Loss: 2.135287
Epoch: 6 - Loss: 2.228964
Epoch: 6 - Loss: 2.164155
Epoch: 6 - Loss: 2.261176
Epoch: 7 - Loss: 1.888180
Epoch: 7 - Loss: 1.870204
Epoch: 7 - Loss: 2.017074
Epoch: 8 - Loss: 1.723309
Epoch: 8 - Loss: 1.835255
Epoch: 8 - Loss: 2.095014
Epoch: 9 - Loss: 1.831653
Epoch: 9 - Loss: 2.085825
Epoch: 9 - Loss: 1.689858
Epoch: 10 - Loss: 1.731391
Epoch: 10 - Loss: 1.635352
Epoch: 10 - Loss: 1.922430
Epoch: 11 - Loss: 1.766142
Epoch: 11 - Loss: 1.916741
Epoch: 11 - Loss: 1.480135
Epoch: 12 - Loss: 1.570211
Epoch: 12 - Loss: 1.574466
Epoc

KeyboardInterrupt: 

In [123]:
test_transforms = A.Compose([
    A.Normalize(),
    ToTensor()
])



test_dataset = CustomDataset(test,'test/test','test',test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=0)

In [124]:
model.eval()
y_pred = list()
with torch.no_grad():
    for i, d in enumerate(test_loader):
        inputs = d.to('cuda')
        outputs = model(inputs)
        y_pred.append(torch.argmax(outputs, -1).cpu().numpy())
y_pred = np.concatenate(y_pred, 0)

In [125]:
submission = pd.read_csv('sample_submission.csv')
submission['label'] = y_pred

In [126]:
submission.to_csv('sub_211117.csv',index = False)